In [2]:
## load libraries 
import os
from dotenv import load_dotenv 
import numpy as np
from typing import List, Dict, Any, Optional
import warnings
warnings.filterwarnings("ignore")

#langchain_core imports 
from langchain_core.documents import Document 
from langchain_core.runnables import RunnableAssign, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Langchain specific imports

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader

In [3]:
load_dotenv(override=True)

True

In [4]:
sample_documents = [
    Document(
        page_content="""
        Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
        """,
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"}
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='\n        Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.\n        '), Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '), Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolu

In [5]:
### text splitter 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators = [" "]
)

##split the documents into chunks 
chunks = text_splitter.split_documents(sample_documents)
print(len(chunks))

4


In [6]:
### load the embedding models 
embeddings = OpenAIEmbeddings(
model="text-embedding-3-small",
)


In [7]:
sample_embedding = embeddings.embed_query("my shoes are black")
print(len(sample_embedding))

1536


In [8]:
word_embeddings = embeddings.embed_documents(["The sky is blue", "I love the blue sky", "The moon looks good today"])

print(f"we have embeddings for {len(word_embeddings)} of dimension: {len(word_embeddings[0])}")

we have embeddings for 3 of dimension: 1536


#### Create FAISS Vector Store 

In [9]:
vectorstore = FAISS.from_documents(
    embedding=embeddings,
    documents=chunks
)

print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 4 vectors


In [10]:
vectorstore.save_local("faiss_index")

print("Vector saved to 'faiss_index' directory")

Vector saved to 'faiss_index' directory


In [11]:
## load_vectorstore

loaded_vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

In [12]:
query = "What is Deep Learning?"

results = vectorstore.similarity_search(query, k=3)

In [13]:
for doc in results:
    print(doc.page_content)
    print("\n")

Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.


Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.


Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.




In [14]:
filter_dict = {"topic":"ML"}

filtered_results = vectorstore.similarity_search_with_score(
    query,
    k=3, 
    filter=filter_dict
)

print(filtered_results)

[(Document(id='e83bd58b-1235-4718-8ec8-b4b2c077a31c', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'), np.float32(1.0857663))]


In [15]:
for doc, score in filtered_results:
    print(f"Score for the document: {score: .4f}")
    print(f"\n With Content: {doc.page_content}")
    print("\n")

Score for the document:  1.0858

 With Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.




In [16]:
2

2

In [17]:
print("hello")

hello


#### Build RAG chain with LCEL

In [18]:
from langchain_groq import ChatGroq

In [19]:
groq_llm = ChatGroq(model="groq/compound-mini", 
                    temperature=0.5,
                    max_retries=2
                    )

In [20]:
groq_llm.invoke("did i connect to you?")

AIMessage(content='Yes—you’re connected and we can chat right now. How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 447, 'total_tokens': 550, 'completion_time': 0.231641, 'prompt_time': 0.02102, 'queue_time': 0.109492, 'total_time': 0.252662}, 'model_name': 'groq/compound-mini', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--0e7ebf72-07a6-4a1d-9811-05b4596c69ea-0', usage_metadata={'input_tokens': 447, 'output_tokens': 103, 'total_tokens': 550})

In [21]:
# For qroq api free tier 
## groq/compound currently has 250 requests per day with no tokens per day limit
##  llama-3.1-8b-instant has 14.4k requests per day wiht a 500k token limit
##  also metalama/llama-4-maverick|scout-17b-128e|1

In [22]:
simple_prompt = ChatPromptTemplate.from_template(
    """ Answer the question based on the following context:
    Context: {context}
    
    Question: {question}
    
    Answer:"""
)

In [23]:
retriever = vectorstore.as_retriever(search_type="similarity",
                        search_kwargs={"k": 3})

In [24]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E2D7BBDA60>, search_kwargs={'k': 3})

In [25]:
def combine_docs(docs: List[Document]) -> str:
    """ Combines all the docs to make the 
    relevant context to be send through RAG chain"""
    all_content = []
    for doc in docs:
        source = doc.metadata.get("source", "unknown")
        all_content.append(f"(Source: {source}):\n{doc.page_content}")

    return "\n\n".join(all_content)

In [26]:
from langchain_core.runnables import RunnableParallel

In [27]:
now_rag_chain = RunnableParallel(
    context = retriever | RunnableLambda(combine_docs),
    question= RunnablePassthrough()
) | simple_prompt | groq_llm | StrOutputParser()

In [28]:
now_rag_chain.invoke("can you explain me about auto encoders in deep learning?")

'**Autoencoders\u202f—\u202fa quick overview**\n\nAn **autoencoder** is a type of artificial neural network used in deep learning for **unsupervised representation learning**. Its goal is to learn a compact (latent) representation of the input data and then reconstruct the original input from that representation.  \n\n---\n\n### 1. Core architecture  \n\n| Component | What it does |\n|-----------|--------------|\n| **Encoder** | Takes the raw input \\(x\\) and maps it to a lower‑dimensional latent vector \\(z = f_{\\text{enc}}(x)\\). This “bottleneck” forces the network to capture the most salient features. |\n| **Latent space (bottleneck)** | The compressed code that ideally contains the essential information about the input while discarding redundancy and noise. |\n| **Decoder** | Reconstructs the input from the latent code \\( \\hat{x} = f_{\\text{dec}}(z) \\). |\n\nDuring training the network minimizes a **reconstruction loss** (e.g., mean‑squared error for images, binary cross‑ent